# Packages

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

# Cloud-Free Composites

In [3]:
Map = geemap.Map()
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', 'Netherlands')
)

Map.addLayer(fc, {'color': 'ff000000'}, "Netherlands")
Map.centerObject(fc)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
start_year = 2013
end_year = 2021

years = ee.List.sequence(start_year, end_year)

def yearly_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, "year")

    collection = (
        ee.ImageCollection('LANDSAT/LC08/C01/T1')
        .filterDate(start_date, end_date)
        .filterBounds(fc)
    )

    image = ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(fc)

    return image

images = years.map(yearly_image)
images

In [5]:
vis_params = {'bands': ['B5', 'B4', 'B3'], 'max': 128}
for index in range(start_year, end_year + 1):
    image = ee.Image(images.get(index - start_year))
    layer_name = "Year " + str(index)
    Map.addLayer(image, vis_params, layer_name)

Map

Map(center=[52.130619595531755, 5.29281217801067], controls=(WidgetControl(options=['position', 'transparent_b…

# Creating timeseries

In [6]:
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

start_date = '2016-01-01'
end_date = '2023-12-31'
region = ee.Geometry.BBox(-122.5549, 37.6968, -122.3446, 37.8111)

images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency = 'year', reducer = 'median'
)
images.size().getInfo()

8

In [7]:
Map = geemap.Map()

vis_params = {
    'min': 0,
    'max': 4000,
    'bands': ['B8', 'B4', 'B3']
}
labels = [str(y) for y in range(2016, 2023 + 1)]

Map.addLayer(images, vis_params, 'Sentinel-2', False)
Map.add_time_slider(images, vis_params, time_interval = 2, labels = labels)
Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## NAIP Timelapse

In [8]:
Map = geemap.Map(center = [40, -100], zoom = 4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [9]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(
    -99.1019, 47.1274, -99.0334, 47.1562,
    # 120.9, 14.3, 121.2, 14.7
    )
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [10]:
collection = geemap.naip_timeseries(roi, start_year = 2009, end_year = 2022, RGBN = True)
collection

In [11]:
years = geemap.image_dates(collection, date_format = 'YYYY').getInfo()
years

['2009',
 '2010',
 '2012',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [12]:
size = len(years)
images = collection.toList(size)
for i in range(size):
    image = ee.Image(images.get(i))
    Map.addLayer(image, {'bands': ['N', 'R', 'G']}, years[i])
Map

Map(center=[47.14180000062306, -99.06765000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
timelapse = geemap.naip_timelapse(
    roi,
    out_gif = 'naip.gif',
    bands = ['N', 'R', 'G'],
    frames_per_second = 3,
    title = 'NAIP Timelapse'
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\naip.gif
ffmpeg is not installed on your computer.


Output()

## Landsat timelapse

In [14]:
Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-74.7222, -8.5867, -74.1596, -8.2824)
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif = 'landsat.gif',
    start_year = 1984,
    end_year = 2023,
    start_date = '01-01',
    end_date = '12-31',
    bands = ['SWIR1', 'NIR', 'Red'],
    frames_per_second = 5,
    title = 'Landsat Timelapse',
    progress_bar_color = 'blue',
    mp4 = True
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\landsat.gif
ffmpeg is not installed on your computer.
ffmpeg is not installed on your computer.


Output()

In [16]:
Map = geemap.Map()
roi = ee.Geometry.BBox(-115.5541, 35.8044, -113.9035, 36.5581)
Map.addLayer(roi)
Map.centerObject(roi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [17]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='las_vegas.gif',
    start_year=1984,
    end_year=2022,
    bands=['NIR', 'Red', 'Green'],
    frames_per_second=5,
    title='Las Vegas, NV',
    font_color='blue',
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\las_vegas.gif
ffmpeg is not installed on your computer.


Output()

In [18]:
# Hong Kong
Map = geemap.Map()
roi = ee.Geometry.BBox(113.8252, 22.1988, 114.0851, 22.3497)
Map.addLayer(roi)
Map.centerObject(roi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [19]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif = 'hong_kong.gif',
    start_year = 1990,
    end_year = 2022,
    start_date = '01-01',
    end_date = '12-31',
    bands = ['SWIR1', 'NIR', 'Red'],
    frames_per_second = 3,
    title = 'Hong Kong'
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\hong_kong.gif
ffmpeg is not installed on your computer.


Output()

In [20]:
# Metro Manila
Map = geemap.Map()
roi = ee.Geometry.BBox(120.9, 14.3, 121.2, 14.8)
Map.addLayer(roi)
Map.centerObject(roi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [21]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif = 'metro_manila.gif',
    start_year = 1987,
    end_year = 2023,
    start_date = '01-01',
    end_date = '12-31',
    bands = ['SWIR1', 'NIR', 'Red'],
    frames_per_second = 2,
    title = 'Metro Manila'
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\metro_manila.gif
ffmpeg is not installed on your computer.


Output()

## Sentinel-1 Timelapse

In [22]:
Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [23]:
timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif = 'sentinel1.gif',
    start_year = 2019,
    end_year = 2019,
    start_date = '04-01',
    end_date = '08-01',
    frequency = 'day',
    vis_params = {'min': -30, 'max': 0},
    palette = 'Greys',
    frames_per_second = 3,
    title = 'Sentinel-1 Timelapse',
    add_colorbar = True,
    colorbar_bg_color = 'gray'
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\sentinel1.gif
ffmpeg is not installed on your computer.


Output()

## Sentinel-2 Timelapse

In [24]:
Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-74.7222, -8.5867, -74.1596, -8.2824)
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [25]:
timelapse = geemap.sentinel2_timelapse(
    roi,
    out_gif = 'sentinel2.gif',
    start_year = 2016,
    end_year = 2022,
    start_date = '01-01',
    end_date = '12-31',
    frequency = 'year',
    bands = ['SWIR1', 'NIR', 'Red'],
    frames_per_second = 5,
    title = 'Sentinel-2 Timelapse',
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\sentinel2.gif
ffmpeg is not installed on your computer.


Output()

## MODIS timelapse

### NDVI

In [26]:
Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-18.6983, -36.1630, 52.2293, 38.1446)
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [27]:
timelapse = geemap.modis_ndvi_timelapse(
    roi,
    out_gif = 'ndvi.gif',
    data = 'Terra',
    band = 'NDVI',
    start_date = '2000-01-01',
    end_date = '2022-12-31',
    frames_per_second = 3,
    title = 'MODIS NDVI Timelapse',
    overlay_data = 'countries'
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\ndvi.gif
ffmpeg is not installed on your computer.


Output()

### Temperature

In [28]:
Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-171.21, -57.13, 177.53, 79.99)
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [29]:
timelapse = geemap.modis_ocean_color_timelapse(
    satellite = 'Aqua',
    start_date = '2018-01-01',
    end_date = '2022-12-31',
    roi = roi,
    frequency = 'month',
    out_gif = 'temperature.gif',
    overlay_data = 'continents',
    overlay_color = 'yellow',
    overlay_opacity = 0.5
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\temperature.gif
ffmpeg is not installed on your computer.


Output()

## GOES timelapse

In [30]:
# Volcano
Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(167.1898, -28.5757, 202.6258, -12.4411)
    Map.addLayer(roi)
    Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [31]:
start_date = "2022-01-15T02:00:00"
end_date = "2022-01-15T08:00:00"
data = "GOES-17"
scan = "full_disk"

timelapse = geemap.goes_timelapse(
    roi,
    'goes.gif',
    start_date,
    end_date,
    data,
    scan,
    framesPerSecond = 4
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\goes.gif
ffmpeg is not installed on your computer.


Output()

In [32]:
# Hurricane
roi = ee.Geometry.BBox(-159.5954, 24.5178, -114.2438, 60.4088)
start_date = "2021-10-24T14:00:00"
end_date = "2021-10-25T01:00:00"
data = "GOES-17"
scan = "full_disk"

In [33]:
timelapse = geemap.goes_timelapse(
    roi,
    'data/hurricane.gif',
    start_date,
    end_date,
    data,
    scan,
    framesPerSecond = 4
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\hurricane.gif
ffmpeg is not installed on your computer.


Output()

In [34]:
# Fire
Map = geemap.Map()
roi = ee.Geometry.BBox(-121.0034, 36.8488, -117.9052, 39.0490)
Map.addLayer(roi)
Map.centerObject(roi)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [35]:
start_date = "2020-09-05T15:00:00"
end_date = "2020-09-06T02:00:00"
data = "GOES-17"
scan = "full_disk"

timelapse = geemap.goes_fire_timelapse(
    roi,
    'data/fire.gif',
    start_date,
    end_date,
    data,
    scan,
    framesperSecond = 5
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\fire.gif
ffmpeg is not installed on your computer.


Output()

# Adding Animated Text

## Add animated text to an existing GIF

In [36]:
url = 'https://github.com/gee-community/geemap/raw/master/examples/data/animation.gif'
filename = 'data/animation.gif'
geemap.download_file(url, filename)

geemap.show_image(url, filename)

data/animation.gif already exists. Skip downloading. Set overwrite=True to overwrite.


Output()

In [37]:
# Add animated text to GIF
out_gif = 'data/timelapse.gif'
geemap.add_text_to_gif(
    filename,
    out_gif,
    xy = ('5%', '5%'),
    text_sequence = 1984,
    font_size = 30,
    font_color = '#0000ff',
    duration = 100
)

geemap.show_image(out_gif)

Output()

In [38]:
# Adding a place name
geemap.add_text_to_gif(
    out_gif, out_gif, xy = ('35%', '85%'), text_sequence = 'Las Vegas', font_color = 'black'
)
geemap.show_image(out_gif)

Output()

## Create GIF from EE data

In [39]:
# Define an area of interest geometry with a global non-polar extent
aoi = ee.Geometry.Polygon(
    [[[-179.0, 78.0], [-179.0, -58.0], [179.0, -58.0], [179.0, 78.0]]], None, False
)

# Import hourly predicted temperature image collection for the northern winter solstice
tempCol = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2018-12-22', '2018-12-23')
    .limit(24)
    .select('temperature_2m_above_ground')
)
# Note that predictions extend for 384 hours, so we limit the collection to the first 24

aoi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -179.0,
              78.0
            ],
            [
              -179.0,
              -58.0
            ],
            [
              179.0,
              -58.0
            ],
            [
              179.0,
              78.0
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [40]:
# Define arguments for animation function parameters
videoArgs = {
    'dimensions': 768,
    'region': aoi,
    'framesPerSecond': 10,
    'crs': 'EPSG:3857',
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}

In [41]:
# Save GIF to local drive
saved_gif = 'temperature.gif'
geemap.download_ee_video(tempCol, videoArgs, saved_gif)

geemap.show_image(saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\temperature.gif


Output()

In [42]:
# Generate an hourly text sequence
text = [str(n).zfill(2) + ':00' for n in range(0, 24)]
text

['00:00',
 '01:00',
 '02:00',
 '03:00',
 '04:00',
 '05:00',
 '06:00',
 '07:00',
 '08:00',
 '09:00',
 '10:00',
 '11:00',
 '12:00',
 '13:00',
 '14:00',
 '15:00',
 '16:00',
 '17:00',
 '18:00',
 '19:00',
 '20:00',
 '21:00',
 '22:00',
 '23:00']

In [43]:
# Add text to GIF
out_gif = 'output2.gif'

geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy = ('3%', '5%'),
    text_sequence = text,
    font_size = 30,
    font_color = '#ffffff'
)

geemap.show_image(out_gif)

Output()

In [44]:
geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy = ('32%', '92%'),
    text_sequence = 'NOAA GFS Hourly Temperature',
    font_color = 'white'
)

geemap.show_image(out_gif)

Output()

## Adding colorbar to an image

In [45]:
# Download a GIF
url = 'https://i.imgur.com/MSde1om.gif'
in_gif = 'data/temp.gif'
geemap.download_file(url, in_gif)

geemap.show_image(in_gif)

data/temp.gif already exists. Skip downloading. Set overwrite=True to overwrite.


Output()

In [46]:
# Get image URLs
noaa_logo = 'https://bit.ly/3ahJoMq'
ee_logo = 'https://i.imgur.com/Qbvacvm.jpg'

# Set output GIF path
out_gif = 'data/output_colorbar.gif'

# Add images to GIF
# geemap.add_image_to_gif(
#     in_gif, out_gif, in_image=noaa_logo, xy=('2%', '80%'), image_size=(80, 80)
# )
geemap.add_image_to_gif(
    in_gif, out_gif, in_image = ee_logo, xy = ('13%', '79%'), image_size = (85, 85)
)

# Display output GIF
geemap.show_image(out_gif)

Output()

In [47]:
# Create a colorbar
width = 250
height = 30
palette = ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
labels = [-40, 35]

colorbar = geemap.create_colorbar(
    width = width,
    height = height,
    palette = palette,
    vertical = False,
    add_labels = True,
    font_size = 20,
    labels = labels
)

geemap.show_image(colorbar)

Output()

In [48]:
# Add colorbar to GIF
geemap.add_image_to_gif(
    out_gif, out_gif, in_image = colorbar, xy = ('69%', '89%'), image_size = (250, 250)
)
geemap.show_image(out_gif)

Output()

# Visualizing Global Land Cover Data Products

## Creating Dynamic World Land Cover Composites

- App: https://www.dynamicworld.app
- App2: https://earthoutreach.users.earthengine.app/view/dynamicworld
- Paper: https://doi.org/10.1038/s41597-022-01307-4
- Model: https://github.com/google/dynamicworld
- Training data: https://doi.pangaea.de/10.1594/PANGAEA.933475
- Data: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1
- JavaScript tutorial: https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1

In [49]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [50]:
# Set the region of interest by drawing a polygon
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)
    Map.addLayer(region, {}, 'Region')

Map.centerObject(region)

In [51]:
# Set the date range
start_date = '2021-01-01'
end_date = '2023-01-01'

# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date, clip = True)
vis_params = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 3000
}
Map.addLayer(image, vis_params, 'Sentinel-2 image')
Map

Map(center=[43.05865000086152, -89.38675], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [52]:
# Create Dynamic World Land cover composite
landcover = geemap.dynamic_world(
    region,
    start_date,
    end_date,
    clip = True,
    return_type = 'class'
)

dwVisParams = {
    'min': 0,
    'max': 8,
    'palette': [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

Map.addLayer(landcover, dwVisParams, 'Land Cover Class')
Map

Map(center=[43.05865000086152, -89.38675], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [53]:
Map.add_legend(title = 'Dynamic World Land Cover', builtin_legend = 'Dynamic_World')

landcover = geemap.dynamic_world(
    region,
    start_date,
    end_date,
    clip = True,
    return_type = 'visualize'
)

Map.addLayer(landcover, {}, 'Land Cover Visualize')
Map

Map(center=[43.05865000086152, -89.38675], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [54]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(
    region,
    start_date,
    end_date,
    clip = True,
    return_type = 'hillshade'
)

Map.addLayer(landcover, {}, 'Land Cover')
Map

Map(center=[43.05865000086152, -89.38675], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

## Comparing Global Land Cover Data Products

In [55]:
# Visualizing ESA Global Land Cover
start_date = '2021-01-01'
end_date = '2022-01-01'
region = ee.Geometry.BBox(-179, -89, 179, 89)

Map = geemap.Map()

esa = ee.ImageCollection('ESA/WorldCover/v200').first()
esa_vis = {'bands': ['Map']}

Map.addLayer(esa, esa_vis, 'ESA LC 2021')
Map.add_legend(title = 'ESA Land Cover', builtin_legend = 'ESA_WorldCover')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [56]:
# Visualizing ESRI Global Land Cover
Map = geemap.Map()

esri = (
    ee.ImageCollection(
        "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
    )
    .filterDate(start_date, end_date)
    .mosaic()
)

esri_vis = {
    'min': 1,
    'max': 11,
    'palette': [
        "#1A5BAB",
        "#358221",
        "#000000",
        "#87D19E",
        "#FFDB5C",
        "#000000",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
        "#C6AD8D",
    ],
}

Map.addLayer(esri, esri_vis, 'ESRI LC 2021')
Map.add_legend(title = 'ESRI Land Cover', builtin_legend = 'ESRI_LandCover_TS')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [57]:
# Dynamic
Map = geemap.Map()

dw_class = geemap.dynamic_world(region, start_date, end_date, return_type = 'class')
dw = geemap.dynamic_world(region, start_date, end_date, return_type = 'hillshade')

dw_vis = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

Map.addLayer(dw_class, dw_vis, 'DW LC 2021', False)
Map.addLayer(dw, {}, 'DW LC Hillshade')

Map.add_legend(title = "Dynamic World Land Cover", builtin_legend = 'Dynamic_World')
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [58]:
# Comparing Dynamic World and ESA Land Cover
Map = geemap.Map(center=[39.3322, -106.7349], zoom=10)

left_layer = geemap.ee_tile_layer(esa, esa_vis, "ESA LC")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World LC")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESA Land Cover", builtin_legend='ESA_WorldCover', position='bottomleft'
)
Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend='Dynamic_World',
    position='bottomright',
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [59]:
# Comparing Dynamic World with ESRI Land Cover
Map = geemap.Map(center=[-89.3998, 43.0886], zoom=10)

left_layer = geemap.ee_tile_layer(esri, esri_vis, "ESRI LC")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World LC")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESRI Land Cover", builtin_legend='ESRI_LandCover_TS', position='bottomleft'
)
Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend='Dynamic_World',
    position='bottomright',
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Creating Dynamic World Time Series

In [60]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [61]:
# Set the region of interest by simply drawing a polygon
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)
    Map.addLayer(region, {}, 'Region')

Map.centerObject(region)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [62]:
# Set the date range
start_date = '2017-01-01'
end_date = '2024-01-01'

images = geemap.dynamic_world_timeseries(
    region, start_date, end_date, frequency = 'year', return_type = 'class'
)

vis_params = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}
Map.addLayer(images.first(), vis_params, 'First image')
Map.add_legend(title = "Dynamic World Land Cover", builtin_legend = 'Dynamic_World')

Map

Map(center=[43.05865000086152, -89.38675], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [63]:
Map.ts_inspector(images, left_vis = vis_params, date_format = 'YYYY')

In [64]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(region)

images = geemap.dynamic_world_timeseries(
    region, start_date, end_date, frequency = 'year', return_type = "hillshade"
)
Map.ts_inspector(images, date_format = 'YYYY')
Map.add_legend(title = "Dynamic World Land Cover", builtin_legend = 'Dynamic_World')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Layo…

# Analyzing Global Land Cover Data

In [65]:
Map = geemap.Map()
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()

Map.addLayer(dataset, {'bands': ['Map']}, 'ESA Land Cover')
Map.add_legend(title = 'ESA Land Cover Type', builtin_legend = 'ESA_WorldCover')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [66]:
df = geemap.image_area_by_group(
    dataset,
    scale = 1000,
    denominator = 1e6,
    decimal_places = 6,
    verbose = True
)
df

Calculating area for group 10 ...
Calculating area for group 20 ...
Calculating area for group 30 ...
Calculating area for group 40 ...
Calculating area for group 50 ...
Calculating area for group 60 ...
Calculating area for group 70 ...
Calculating area for group 80 ...
Calculating area for group 90 ...
Calculating area for group 95 ...
Calculating area for group 100 ...


,area,percentage
group,,
10,4.585214e+07,0.300937
20,9.607911e+06,0.063059
30,3.183386e+07,0.208932
40,1.307032e+07,0.085783
50,7.794734e+05,0.005116
60,2.234225e+07,0.146637
70,2.469120e+06,0.016205
80,2.093868e+07,0.137425
90,2.123103e+06,0.013934


In [67]:
df.to_csv('data/esa2021.csv')

esa_dict = {
    "10 Trees": "006400",
    "20 Shrubland": "ffbb22",
    "30 Grassland": "ffff4c",
    "40 Cropland": "f096ff",
    "50 Built-up": "fa0000",
    "60 Barren / sparse vegetation": "b4b4b4",
    "70 Snow and ice": "f0f0f0",
    "80 Open water": "0064c8",
    "90 Herbaceous wetland": "0096a0",
    "95 Mangroves": "00cf75",
    "100 Moss and lichen": "fae6a0",
}
classes = list(esa_dict.keys())
classes

['10 Trees',
 '20 Shrubland',
 '30 Grassland',
 '40 Cropland',
 '50 Built-up',
 '60 Barren / sparse vegetation',
 '70 Snow and ice',
 '80 Open water',
 '90 Herbaceous wetland',
 '95 Mangroves',
 '100 Moss and lichen']

In [68]:
df['class'] = classes

geemap.bar_chart(
    df,
    x = 'class',
    y = 'area',
    x_label = 'Land Cover Type',
    y_label = 'Area (km2)'
)

In [69]:
geemap.pie_chart(
    df, names = 'class', values = 'area', height = 500
)

In [70]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
Map.addLayer(countries, {}, 'Countries')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [71]:
geemap.zonal_stats_by_group(
    dataset,
    countries,
    'data/esa_2021_country.csv',
    stat_type='SUM',
    denominator=1e6,
    scale=1000,
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\esa_2021_country.csv


In [72]:
geemap.bar_chart(
    'data/esa_2021_country.csv',
    x='NAME',
    y='Class_10',
    max_rows=30,
    x_label='Country',
    y_label='Forest Area (km2)',
)

In [73]:
geemap.bar_chart(
    'data/esa_2021_country.csv',
    x='NAME',
    y='Class_40',
    max_rows=30,
    x_label='Country',
    y_label='Cropland Area (km2)',
)

In [74]:
geemap.bar_chart(
    'data/esa_2021_country.csv',
    x='NAME',
    y=['Class_10', 'Class_20', 'Class_30', 'Class_40'],
    max_rows=10,
    x_label='Country',
)

In [75]:
geemap.pie_chart(
    'data/esa_2021_country.csv', names='NAME', values='Class_10', max_rows=30, height=500
)

In [76]:
geemap.pie_chart(
    'data/esa_2021_country.csv', names='NAME', values='Class_40', max_rows=30, height=500
)

# Forest cover change analysis

In [77]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

dataset = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')
dataset.bandNames().getInfo()

['treecover2000',
 'loss',
 'gain',
 'lossyear',
 'first_b30',
 'first_b40',
 'first_b50',
 'first_b70',
 'last_b30',
 'last_b40',
 'last_b50',
 'last_b70',
 'datamask']

In [78]:
first_bands = ['first_b50', 'first_b40', 'first_b30']
first_image = dataset.select(first_bands)
Map.addLayer(first_image, {'bands': first_bands, 'gamma': 1.5}, 'Year 2000 Bands 5/4/3')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [79]:
last_bands = ['last_b50', 'last_b40', 'last_b30']
last_image = dataset.select(last_bands)
Map.addLayer(last_image, {'bands': last_bands, 'gamma': 1.5}, 'Year 2021 Bands 5/4/3')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [80]:
treecover = dataset.select(['treecover2000'])

treeCoverVisParams = {
    'min': 0,
    'max': 100,
    'palette': ['black', 'green']
}

name1 = 'Tree cover (%)'
Map.addLayer(treecover, treeCoverVisParams, name1)
Map.add_colorbar(treeCoverVisParams, label = name1, layer_name = name1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [81]:
threshold = 10
treecover_bin = treecover.gte(threshold).selfMask()
treeVisParams = {'palette': ['green']}

Map.addLayer(treecover_bin, treeVisParams, 'Tree cover bin')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [82]:
treeloss_year = dataset.select(['lossyear'])
treeLossVisParams = {'min': 0, 'max': 21, 'palette': ['yellow', 'red']}

layer_name = 'Tree loss year'
Map.addLayer(treeloss_year, treeLossVisParams, layer_name)
Map.add_colorbar(treeLossVisParams, label = layer_name, layer_name = layer_name)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [83]:
treeloss = dataset.select(['loss']).selfMask()
Map.addLayer(treeloss, {'palette': 'red'}, 'Tree loss')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [84]:
treegain = dataset.select(['gain']).selfMask()
Map.addLayer(treegain, {'palette': 'yellow'}, 'Tree gain')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [85]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
geemap.ee_to_df(countries)
style = {'color': '#ffff0088', 'fillColor': '#00000000'}

Map.addLayer(countries.style(**style), {}, 'Countries')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Forest area analysis

In [86]:
geemap.zonal_stats_by_group(
    treecover_bin,
    countries,
    'data/forest_cover.csv',
    stat_type = 'SUM',
    denominator = 1e6,
    scale = 1000
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\forest_cover.csv


In [87]:
geemap.pie_chart(
    'data/forest_cover.csv', name = 'NAME', values = 'Class_sum', max_rows = 20, height = 600
)

Exception: Could not create pie chart. pie() got an unexpected keyword argument 'name'

In [ ]:
geemap.bar_chart(
    'data/forest_cover.csv',
    x = 'NAME',
    y = 'Class_sum',
    max_rows = 20,
    x_label = 'Country',
    y_label = 'Forest area (km2)'
)

## Forest loss analysis

In [ ]:
geemap.zonal_stats_by_group(
    treeloss,
    countries,
    'data/treeloss.csv',
    stat_type = 'SUM',
    denominator = 1e6,
    scale = 1000
)

In [ ]:
geemap.pie_chart(
    'data/treeloss.csv', name = 'NAME', values = 'Class_sum', max_rows = 20, height = 600
)

In [ ]:
geemap.bar_chart(
    'data/treeloss.csv',
    x = 'NAME',
    y = 'Class_sum',
    max_rows = 20,
    x_label = 'Country',
    y_label = 'Forest loss area (km2)'
)

# Surface water change analysis

## Surface water occurrence

In [ ]:
dataset = ee.Image('JRC/GSW1_3/GlobalSurfaceWater')
dataset.bandNames().getInfo()

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

image = dataset.select(['occurrence'])
region = ee.Geometry.BBox(-99.957, 46.8947, -99.278, 47.1531)

vis_params = {'min': 0.0, 'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']}

Map.addLayer(image, vis_params, 'Occurrence')
Map.addLayer(region, {}, 'ROI', True, 0.5)
Map.centerObject(region)
Map.add_colorbar(vis_params, label = 'Water occurrence (%)', layer_name = 'Occurrence')

Map

In [ ]:
hist = geemap.image_histogram(
    image,
    region,
    scale = 30,
    x_label = 'Frequency',
    y_label = 'Pixel Count',
    title = 'Water Occurrence',
    return_df = False
)
hist

## Surface water monthly history

In [ ]:
dataset = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory')
size = dataset.size()
size.getInfo()

In [ ]:
dataset.aggregate_array('system:index').getInfo()

In [ ]:
Map = geemap.Map()

image = dataset.filterDate('2020-08-01', '2020-09-01').first()
region = ee.Geometry.BBox(-99.957, 46.8947, -99.278, 47.1531)

vis_params = {
    'min': 0,
    'max': 2.0,
    'palette': ['ffffff', 'fffcb8', '0905ff']
}

Map.addLayer(image, vis_params, 'Water')
Map.addLayer(region, {}, 'ROI', True, 0.5)
Map.centerObject(region)

Map

In [ ]:
jrc = geemap.jrc_hist_monthly_history(
    region = region,
    scale = 30,
    frequency = 'month',
    denominator = 1e4,
    return_df = True
)
jrc

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region, scale=30, frequency='month', denominator=1e4, y_label='Area (ha)'
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency='month',
    y_label='Area (ha)',
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency='month',
    y_label='Area (ha)',
    color='month',
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency='year',
    reducer='mean',
    y_label='Area (ha)',
)

In [ ]:
geemap.jrc_hist_monthly_history(
    region=region,
    start_month=6,
    end_month=9,
    scale=30,
    frequency='year',
    reducer='max',
    y_label='Area (ha)',
)